<h1 style="text-align:center; color:green; font-size:48px;">
OEMOF TUTORIAL
</h1>

### Support functions

In [1]:
# Levelized Cost of Heat

def LCOH(invest_cost, operation_cost, heat_produced, revenue=0, i=0.05, n=20):
    pvf = ((1 + i) ** n - 1) / ((1 + i) ** n * i)

    return (invest_cost + pvf * (operation_cost - revenue)) / (
        pvf * heat_produced
    )
    
# Equivalent Periodic Cost

def epc(invest_cost, i=0.05, n=20):
    af = (i * (1 + i) ** n) / ((1 + i) ** n - 1)

    return invest_cost * af

# Import libraries

In [2]:
import os
import warnings
import logging
import pandas as pd
import matplotlib.pyplot as plt
from oemof.solph import (Bus, EnergySystem, Flow, Model, create_time_index, processing)
from oemof.solph.components import (Sink, Source, Converter, GenericStorage)
from oemof.solph import EnergySystem
from oemof.solph import views
import oemof.solph as solph

# Example

<img src="Example_OEMOF.png" width="40%">

## 1.1 Create the energy system

In [3]:
# read the input data file
filename = r"STEP_3/inputs/data.csv"
data = pd.read_csv(filename, sep=";")

filename2 = r"STEP_3/inputs/irradiance_riga.csv"
irradiance_data = pd.read_csv(filename2, sep=";")
irradiance_data = irradiance_data.fillna(0)

# specifying the solver
solver = "cbc"
solver_verbose = False

# Create energy system
datetimeindex = create_time_index(2022, number=len(data))
energysystem = EnergySystem(timeindex=datetimeindex, infer_last_interval=False)

# --- Step 2: Buses, Sources, Sinks (Single Thermal Bus) ---

# Buses
electrical_bus = Bus(label="electrical_bus")
thermal_bus = Bus(label="thermal_bus")
gas_bus = Bus(label="gas_bus")
biomass_bus = Bus(label="biomass_bus") # New Biomass Bus
waste_heat_bus = Bus(label="waste_heat_bus") # New Waste Heat Bus
ambient_heat_bus = Bus(label="ambient_heat_bus") # for solar collector

energysystem.add(electrical_bus, thermal_bus, gas_bus, biomass_bus, waste_heat_bus, ambient_heat_bus)

# Excess electricity sink
energysystem.add(
    Sink(
        label="excess_electricity",
        inputs={electrical_bus: Flow(variable_costs=data["electricity_price"] * -1)}
    )
)

# Gas source with cost
energysystem.add(
    Source(label="natural_gas",outputs={gas_bus: Flow(variable_costs=(data["gas_price"] + data["co2_price"]) / 1000)})
                )

## Biomass source (Unlimited supply)
# Cost is fixed per unit of heat produced by CHP2, so we set source cost to 0 here.
energysystem.add(
    Source(label="biomass_source", outputs={biomass_bus: Flow(variable_costs=0)})
)

# Waste Heat Source (from data)
# Using max constraint based on data profile
max_waste_heat = data["waste_heat"].max()
if max_waste_heat > 0:
    waste_heat_profile = data["waste_heat"] / max_waste_heat
else:
    waste_heat_profile = [0] * len(data)
    max_waste_heat = 0

energysystem.add(
    Source(
        label="waste_heat_source", 
        outputs={waste_heat_bus: Flow(max=waste_heat_profile, nominal_value=max_waste_heat)}
    )
)

# Grid electricity source
energysystem.add(
    Source(label="electricity_grid", outputs={electrical_bus: Flow(variable_costs=data["electricity_price"])})
)

# Ambient Heat source (Input to Solar Collector)
energysystem.add(
    Source(label="ambient_heat", outputs={ambient_heat_bus: Flow(variable_costs=0)})
)

# Thermal demand sink
max_demand = data["thermal_demand"].max()
demand_fix = data["thermal_demand"] / max_demand

energysystem.add(
    Sink(
        label="thermal_demand",
        inputs={thermal_bus: Flow(
            nominal_value=353e3,
            fix=demand_fix
        )},
    )
)

# **Thermal storage is NOT included (as per Step 3 requirement).**
# Shortage Source (to prevent infeasibility)
energysystem.add(
    Source(
        label="thermal_shortage",
        outputs={thermal_bus: Flow(variable_costs=10000)} # High cost to discourage use
    )
)


# --- Step 3: Heat Producers (5 Total + Solar) ---

# 1. Combined heat and power plant (CHP 1) - fuel 1 (Gas) - Unchanged
energysystem.add(
    Converter(
        label="chp_1", # Renamed to match summary labels
        inputs={gas_bus: Flow(nominal_value=475e3)},
        outputs={
            electrical_bus: Flow(variable_costs=5),
            thermal_bus: Flow()
        },
        conversion_factors={electrical_bus: 0.421, thermal_bus: 0.474}
    )
)

# 2. Combined heat and power plant (CHP 2) - fuel 2 (Biomass)
# Uses unlimited biomass. Fixed cost per unit of heat.
# Efficiency: Kept same as before (0.421 el, 0.474 th) as not specified.
# Fixed cost per unit of heat: Added variable_costs=20 (placeholder) to thermal_bus output.
energysystem.add(
    Converter(
        label="chp_2",
        inputs={biomass_bus: Flow(nominal_value=225e3)}, # Now uses biomass
        outputs={
            electrical_bus: Flow(variable_costs=5),
            thermal_bus: Flow(variable_costs=20) # Fixed cost per unit of heat
        },
        conversion_factors={electrical_bus: 0.421, thermal_bus: 0.474}
    )
)

# 3. Gas Boiler 1 - Unchanged
energysystem.add(
    Converter(
        label="gas_boiler_1",
        inputs={gas_bus: Flow(nominal_value=50e3)},
        outputs={thermal_bus: Flow()},
        conversion_factors={thermal_bus: 0.95}
    )
)

# 4. Heat Pump (MODELED AS A CONVERTER)  - waste heat source
COP = 3.0
# Inputs: Electricity and Waste Heat.
# Output: Heat.
# Relation: Heat = Elec * COP.
# Energy Balance: Heat = Elec + WasteHeat.
# WasteHeat = Heat - Elec = Heat - Heat/COP = Heat * (1 - 1/COP).
# So for 1 unit of Heat output:
# Elec input = 1/COP.
# WasteHeat input = 1 - 1/COP.
# With COP=3: Elec=1/3, Waste=2/3.
# Conversion factors relative to thermal_bus (output) being 1:
# electrical_bus: 1/COP
# waste_heat_bus: (COP-1)/COP
energysystem.add(
    Converter(
        label="heat_pump",
        inputs={
            electrical_bus: Flow(nominal_value=10e3), 
            waste_heat_bus: Flow()
        }, 
        outputs={thermal_bus: Flow()},                     
        conversion_factors={
            thermal_bus: COP, # Output is COP * Elec
            waste_heat_bus: COP - 1 # Input Waste is (COP-1) * Elec
        }
    )
)

# 5. Solar Thermal Collector 
solar_thermal_conversion_factor = 0.5
energysystem.add(
    Converter(
        label="solar_collector",
        inputs={ambient_heat_bus: Flow()},
        outputs={thermal_bus: Flow()},
        conversion_factors={
            # The conversion factor is the time-series irradiance data
            thermal_bus: solar_thermal_conversion_factor * irradiance_data["DNI"]
        },
    )
)

# --- Step 4: Optimization and Results ---

print("Solving Optimization Model...")
model = Model(energysystem)

# Solve the optimization problem
try:
    model.solve(solver=solver, cmdline_options={"mipgap": 0.005}, solve_kwargs={"tee": solver_verbose})
    print("\nOptimization Complete.")
    
    # --- Results Summary ---
    print(f"Total Objective Value (Minimised System Costs): {model.objective():,.2f} EUR")
    results = processing.results(model)
    data_flow = processing.convert_keys_to_strings(results)

    print("\n### Heat Producer Output Summary (Total Flow to thermal_bus)")
    producer_labels = ["chp_1", "chp_2", "gas_boiler_1", "heat_pump", "solar_collector", "thermal_shortage"]
    summary_data = {}

    for label in producer_labels:
        # Correctly construct tuple key
        flow_key = (label, "thermal_bus")
        
        if flow_key in data_flow and 'sequences' in data_flow[flow_key] and 'flow' in data_flow[flow_key]['sequences']:
            flow_sum = data_flow[flow_key]['sequences']['flow'].sum()
            summary_data[label] = flow_sum
        else:
            summary_data[label] = 0.0 

    # Display results
    for producer, flow in sorted(summary_data.items(), key=lambda item: item[1], reverse=True):
        print(f"- {producer.replace('_', ' ').title():<18}: {flow:,.2f} MWh")

except Exception as e:
    print(f"\nOptimization Failed. Check the solver installation or data integrity. Error: {e}")


Solving Optimization Model...

Optimization Complete.
Total Objective Value (Minimised System Costs): -94,808,506,461.35 EUR

### Heat Producer Output Summary (Total Flow to thermal_bus)
- Chp 1             : 1,196,682,132.24 MWh
- Chp 2             : 53,112,414.46 MWh
- Solar Collector   : 29,008,215.00 MWh
- Gas Boiler 1      : 23,094,635.99 MWh
- Heat Pump         : 6,026.28 MWh
- Thermal Shortage  : 0.00 MWh


## 1.2 Build and solve model

In [4]:
model = Model(energysystem)


logging.info("Solving the optimization problem.")
model.solve(
    solver='cbc',
    solve_kwargs={"tee": True},
    cmdline_options={"ratioGap": "0.02"}
)

'''
This returns a dictionary containing time series and scalar results for all components (buses, converters, sources, etc.).
Or view results for a specific node (for example, a bus).
'''

# Process results
energysystem.results["main"] = processing.results(model)
energysystem.results["meta"] = processing.meta_results(model)

# Save results to file
output_file = os.path.join(os.getcwd(), "STEP_3/outputs/results.oemof")

energysystem.dump(os.getcwd(), "STEP_3/outputs/results.oemof")

logging.info("Results have been dumped.")

Welcome to the CBC MILP Solver 
Version: 2.10.12 
Build Date: Aug  2 2025 

command line - /opt/anaconda3/envs/env_P2/bin/cbc -ratioGap 0.02 -printingOptions all -import /var/folders/48/nvz0qnqd5zd_kdw85vnnhrl40000gn/T/tmp7l7tzu7b.pyomo.lp -stat=1 -solve -solu /var/folders/48/nvz0qnqd5zd_kdw85vnnhrl40000gn/T/tmp7l7tzu7b.pyomo.soln (default strategy 1)
ratioGap was changed from 0 to 0.02
Option for printingOptions changed from normal to all
Presolve 12289 (-110351) rows, 38057 (-137143) columns and 51504 (-257783) elements
Statistics for presolved model


Problem has 12289 rows, 38057 columns (32016 with objective) and 51504 elements
There are 24610 singletons with objective 
Column breakdown:
18155 of type 0.0->inf, 19902 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
6041 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
6041 of type E other, 0 of type G 0.0, 0 of typ

/opt/anaconda3/envs/env_P2/lib/python3.9/site-packages/oemof/network/energy_system.py:256: FutureWarning: Parameter 'dpath' will be removed in a future version. You can give the directory as part of the filename and set 'consider_dpath' to False to suppress this waring.
  warnings.warn(


## 1.3 Results

In [5]:
energysystem.results

results = energysystem.results["main"]

# get all variables of a specific component/bus
thermal_bus = views.node(results, "thermal_bus")
storage = views.node(results, "storage")

### Analyse thermal node

In [6]:
Sum_th_demand = thermal_bus["sequences"][(('thermal_bus', 'thermal_demand'), 'flow')].sum()
Sum_th_prod_chp = thermal_bus["sequences"][(('chp', 'thermal_bus'), 'flow')].sum()
Sum_th_prod_pth = thermal_bus["sequences"][(('electric_boiler', 'thermal_bus'), 'flow')].sum()

fig, axs = plt.subplots(3,figsize=(16, 9))
fig.suptitle('Thermal flows comparison', fontsize=24)
axs[0].plot(thermal_bus["sequences"][(('thermal_bus', 'thermal_demand'), 'flow')], 'blue')
axs[0].set_title(f'bth to demand_th, Sum = {int(Sum_th_demand/1000)} MWh')
axs[1].plot(thermal_bus["sequences"][(('chp', 'thermal_bus'), 'flow')], 'green')
axs[1].set_title(f'pp_chp to bth, Sum = {int(Sum_th_prod_chp/1000)} MWh')
axs[2].plot(thermal_bus["sequences"][(('electric_boiler', 'thermal_bus'), 'flow')], 'orange')
axs[2].set_title(f'pth to bth, Sum = {int(Sum_th_prod_pth/1000)} MWh')

axs[0].set_ylabel('Power in kW')
axs[1].set_ylabel('Power in kW')
axs[2].set_ylabel('Power in kW')

plt.tight_layout()
plt.show()

KeyError: (('chp', 'thermal_bus'), 'flow')

### Analyse storage node

In [ ]:
# Analyse storage node
# Check if storage exists in the energy system
if storage and 'sequences' in storage:
    Sum_storage_Outflow = storage['sequences'][(('storage', 'thermal_bus'), 'flow')].sum()
    Sum_storage_Inflow = storage['sequences'][(('thermal_bus', 'storage'), 'flow')].sum()
else:
    print("No storage component found in the energy system. Skipping storage analysis.")
    Sum_storage_Outflow = 0
    Sum_storage_Inflow = 0

fig, ax = plt.subplots(figsize=(16, 9))
storage['sequences'][(('thermal_bus', 'storage'), 'flow')].plot(
    ax=ax, kind="line", drawstyle="steps-post",color="green"
)
plt.legend(
    loc="upper right"
)
fig.subplots_adjust(top=0.8)
fig.suptitle(f'Storage inflow, Sum = {int(Sum_storage_Inflow / 1000)} MWh', fontsize=24)
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(16, 9))
storage['sequences'][(('storage', 'thermal_bus'), 'flow')].plot(
    ax=ax, kind="line", drawstyle="steps-post",color="blue"
)
plt.legend(
    loc="upper right"
)
fig.subplots_adjust(top=0.8)
fig.suptitle(f'Storage outflow, Sum = {int(Sum_storage_Outflow / 1000)} MWh', fontsize=24)
plt.show()

fig, ax = plt.subplots(figsize=(16, 9))
storage['sequences'][(('storage', 'None'), 'storage_content')].plot(
    ax=ax, kind="line", drawstyle="steps-post",color="orange"
)
plt.legend(
    loc="upper right"
)
fig.subplots_adjust(top=0.8)
fig.suptitle('Storage content', fontsize=24)
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(3,figsize=(16, 9))
fig.suptitle('Comparison of storage flows and content', fontsize=24)
axs[0].plot(storage['sequences'][(('thermal_bus', 'storage'), 'flow')], 'green')
axs[0].set_title(f'Storage inflow, Sum = {int(Sum_storage_Inflow / 1000)} MWh')
axs[1].plot(storage['sequences'][(('storage', 'thermal_bus'), 'flow')], 'blue')
axs[1].set_title(f'Storage outflow, Sum = {int(Sum_storage_Outflow / 1000)} MWh')
axs[2].plot(storage['sequences'][(('storage', 'None'), 'storage_content')], 'orange')
axs[2].set_title('Storage content')

axs[0].set_ylabel('Power in kW')
axs[1].set_ylabel('Power in kW')
axs[2].set_ylabel('Content in kWh')

plt.tight_layout()
plt.show()

In [ ]:
heat_bus = solph.Bus(label="heat_network")
gas_bus = solph.Bus(label="gas_network")
waste_heat_bus = solph.Bus(label="waste_heat_network")
electrical_bus = solph.Bus(label="electrical_network")

district_heating_demand = solph.components.Sink(
    label="district_heating_demand",
    inputs={heat_bus: solph.Flow(nominal_value=100e3, fix=data["heat_demand"])}
)

energysystem.add(heat_bus, gas_bus, waste_heat_bus, electrical_bus, district_heating_demand)

heat_sink = solph.components.Sink(
    label="heat_sink",
    inputs={heat_bus: solph.Flow(nominal_value=100e3, fix=data["heat_demand"])}
)

energysystem.add(heat_sink)

district_heating_demand = solph.Sink(
    label="district_heating_demand",
    inputs={heat_bus: solph.Flow(nominal_value=100e3, fix=data["heat_demand"])}
)

energysystem.add(heat_bus, gas_bus, waste_heat_bus, electrical_bus, district_heating_demand)

spec_inv_gas_boiler = 60000
var_cost_gas_boiler = 1.1


district_heating_system.add(waste_heat_bus)

gas_boiler = solph.components.Converter(
    label="gas_boiler",
    inputs={gas_bus: solph.Flow()},
    outputs={waste_heat_bus: solph.Flow(nominal_value=100e3, variable_costs=data["gas_price"])}
)
energysystem.add(gas_boiler)
